In [1]:
pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 15.1 MB/s 
     |████████████████████████████████| 163 kB 51.7 MB/s 
     |████████████████████████████████| 7.6 MB 52.5 MB/s 


In [2]:
pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.8 MB/s 
     |████████████████████████████████| 1.3 MB 46.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=8dcff362e65a96d2d14c81ddd2495e26f30f91e730135f038682fb7aadb9ffec
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [3]:
import pandas as pd
from transformers import BertTokenizerFast, BertForSequenceClassification, get_linear_schedule_with_warmup, AdamW
from sklearn.model_selection import train_test_split
import torch
from tqdm import tqdm 
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import os
import numpy as np
import random
from sklearn.metrics import accuracy_score
import json

In [4]:
df = pd.read_csv("/content/drive/MyDrive/train (2).csv")

In [5]:
df.head()

,pair_id,name_1,name_2,is_duplicate
0,1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0
1,2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0
2,3,"Rishichem Distributors Pvt., Ltd.",Dsa,0
3,4,Powermax Rubber Factory,Co. One,0
4,5,Tress A/S,Longyou Industries Park Zhejiang,0


In [6]:
filtered_df_0 = df[df['is_duplicate']==0].sample(4000)
filtered_df_1 = df[df['is_duplicate']==1]
filtered_df = pd.concat([filtered_df_0, filtered_df_1], axis=0)

In [7]:

print(len(df))
print(len(filtered_df))
print(len(filtered_df_0))
print(len(filtered_df_1))
df.head()
filtered_df.head()
filtered_df_1.head()


497819
7658
4000
3658


,pair_id,name_1,name_2,is_duplicate
161,162,JX Nippon Oil & Gas Exploration (Brasil) Ltda,JX Nippon Oil & Gas Exploration Technical Serv...,1
603,604,Pirelli Neumaticos S.A.I.C.,"Pirelli Tyre Co., Ltd.",1
835,836,Brenntag Australia (Pty) Ltd.,Brenntag Group,1
1328,1329,"PAUL BAUDER GMBH & CO KG, BOCHUM PLANT",Paul Bauder ag,1
1562,1563,TOTAL CESKA REPUBLIKA s.r.o.,TOTAL FRANCE (ARNAY LE DUC),1


In [8]:
filtered_df_test = df[df['name_1']=="JX Nippon Oil & Gas Exploration (Brasil) Ltda"]
print(len(filtered_df_test))
filtered_df_test.tail()

10


,pair_id,name_1,name_2,is_duplicate
316074,316075,JX Nippon Oil & Gas Exploration (Brasil) Ltda,JX Nippon Oil & Gas Exploration (Myanmar) Limited,1
328496,328497,JX Nippon Oil & Gas Exploration (Brasil) Ltda,Nippon Oil Exploration U.S.A. Limited,1
389435,389436,JX Nippon Oil & Gas Exploration (Brasil) Ltda,Nippon Oil Exploration Limited,1
395094,395095,JX Nippon Oil & Gas Exploration (Brasil) Ltda,JX Nippon Oil & Gas Exploration (Malaysia) Lim...,1
444191,444192,JX Nippon Oil & Gas Exploration (Brasil) Ltda,JX Nippon Oil & Gas Exploration Corporation,1


In [9]:
# raw_model = 'bert-base-uncased'
# tokenizer = BertTokenizerFast.from_pretrained(raw_model, do_lower_case=True)

In [10]:
def seed_everything(seed = 1234):
     random.seed(seed)
     os.environ['PYTHONHASHSEED'] = str(seed)
     np.random.seed(seed)     
     torch.manual_seed(seed)
     torch.cuda.manual_seed(seed)
     torch.backends.cudnn.deterministic = True

In [11]:
seed_everything()

In [26]:
# tokens = tokenizer('Htl Logistics India Pvt., Ltd.')
# tokens

{'input_ids': [101, 1044, 19646, 12708, 2634, 26189, 2102, 1012, 1010, 5183, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [27]:
# tokens.input_ids

[101, 1044, 19646, 12708, 2634, 26189, 2102, 1012, 1010, 5183, 1012, 102]

In [17]:
# НЕ ЗАПУСКАТЬ ДАННЫЙ СЕЛЛ
triplet_dataset = []
added_examples = {}
neg_max = 10
skip_counter = 0
for i, row in tqdm(filtered_df_1.iterrows()):
  # print(row)
  # print(i)
  if row["is_duplicate"] == 1:
    new_example = {}
    new_example["set"] = row["name_1"]
    new_example["pos"] = row["name_2"]
    new_example["neg"] = []
    neg_counter = 0

    for j, row_2 in df.iterrows():
      if j < skip_counter:
        continue
      if row_2["name_1"] != new_example["set"] and row_2["name_1"] != new_example["pos"] and row_2["name_1"] not in new_example["neg"]:
      #if True:
        new_example["neg"].append(row_2["name_1"])
        #added_examples[row_2["name_1"]] = None
        neg_counter += 1
        if neg_counter >= neg_max:
          skip_counter = j
          neg_counter = 0
          break
    triplet_dataset.append(new_example)
    #print(new_example)

  
json.dump(triplet_dataset, open("triplet_dataset.json", "w"), indent=4)

print(triplet_dataset)

3658it [44:39,  1.37it/s]

[{'set': 'JX Nippon Oil & Gas Exploration (Brasil) Ltda', 'pos': 'JX Nippon Oil & Gas Exploration Technical Services Corporation', 'neg': ['Iko Industries Ltd.', 'Apcotex Industries Ltd.', 'Rishichem Distributors Pvt., Ltd.', 'Powermax Rubber Factory', 'Tress A/S', 'Wpt Mexico Sa De Cv', 'National Bank Of', 'Citibank Korea Inc.', 'Prettl Electric Del Bajio S.A. De C.V.', 'Beijing Zhongyi Rongda Tech Trading Co., Ltd.']}, {'set': 'Pirelli Neumaticos S.A.I.C.', 'pos': 'Pirelli Tyre Co., Ltd.', 'neg': ['Beijing Zhongyi Rongda Tech Trading Co., Ltd.', 'R.I.Intl', 'Angle Industries Ltd.', 'Bestocean Worldwide Logistics Inc.', 'Uni Trans Illinois Consolidated Inc.', 'Warehouse Jonesboro', 'Ktn Underwood Arlanxeo Usa Llc', 'Wynn Express Corp', 'Parker Industrials De Rl De Cv', 'Air Sea Transport (Chicago) Inc.']}, {'set': 'Brenntag Australia (Pty) Ltd.', 'pos': 'Brenntag Group', 'neg': ['Air Sea Transport (Chicago) Inc.', 'Shanghai Scientific Instruments And Materials Co., Ltd.', 'Faj Interna

In [12]:
triplet_dataset = json.load(open("/content/drive/MyDrive/triplet_dataset.json"))

In [23]:
DEVICE = 'cuda:0'

In [25]:
from sentence_transformers import InputExample

train_examples = []
test_examples = []
every_n_test = 10

for i, sample in tqdm(enumerate(triplet_dataset)):
  #example = train_data[i]
  if i % every_n_test == 0:
    test_examples.append(InputExample(texts=[sample['set'], sample['pos'], sample['neg'][0]]))
    test_examples.append(InputExample(texts=[sample['pos'], sample['set'], sample['neg'][1]]))
    continue
  else:
    for j in range(len(sample['neg'])):
      if j % 2 == 0:
        train_examples.append(InputExample(texts=[sample['set'], sample['pos'], sample['neg'][j]]))
      else:
        train_examples.append(InputExample(texts=[sample['pos'], sample['set'], sample['neg'][j]]))
    


3658it [00:00, 5870.53it/s] 


In [14]:
print(f"We have a {type(train_examples)} of length {len(train_examples)} containing {type(train_examples[0])}'s.")

We have a <class 'list'> of length 32920 containing <class 'sentence_transformers.readers.InputExample.InputExample'>'s.


In [15]:
print(f"We have a {type(test_examples)} of length {len(test_examples)} containing {type(test_examples[0])}'s.")

We have a <class 'list'> of length 732 containing <class 'sentence_transformers.readers.InputExample.InputExample'>'s.


In [16]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=)

In [17]:
from sentence_transformers import SentenceTransformer, models

## Step 1: use an existing language model
word_embedding_model = models.Transformer('distilroberta-base')

#word_embedding_model = models.Transformer('stsb-roberta-large')
## Step 2: use a pool function over the token embeddings
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

## Join steps 1 and 2 using the modules argument
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [18]:
from sentence_transformers import losses

train_loss = losses.TripletLoss(model=model)

In [19]:
num_epochs = 5

warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data

In [20]:
DEVICE = 'cuda:0'
model.to(DEVICE)


SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [22]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          warmup_steps=warmup_steps) 

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/258 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
torch.save(model.state_dict(), "model_state")
torch.save(model, "model")

In [ ]:
embeddings = model.encode(sentences, convert_to_tensor=True)

  similarity = []
  for i in range(len(sentences)):
    row = []
    for j in range(len(sentences)):
      row.append(util.pytorch_cos_sim(embeddings[i], embeddings[j]).item())
    similarity.append(row)
    
create_heatmap(similarity)